# Dealing with Pile-up in an EPIC Source
<hr style="border: 2px solid #fadbac" />

- **Description:** Introduction on how to deal with pile-up from a bright source.
- **Level:** Intermediate
- **Data:** XMM observation of Algol (obsid=0112880701)
- **Requirements:** Must be run using pySAS version 2.0.
- **Credit:** Ryan Tanner (March 2025)
- **Support:** <a href="https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html">XMM Newton GOF Helpdesk</a>
- **Last verified to run:** 21 July 2025, for SAS v22.1 and pySAS v2.0

<hr style="border: 2px solid #fadbac" />

## 1. Introduction
This tutorial is a supplement to the introductory notebooks on extracting a source in from EPIC data (XMM-Newton ABC Guide, Chapter 7, [Part 1](./analysis-xmm-ABC-guide-EPIC-image-filtering.ipynb "EPIC Image Filtering") and [Part 2](./analysis-xmm-ABC-guide-EPIC-source-spectrum.ipynb "EPIC Source Extraction")). This notebook assumes you are at least minimally familiar with pySAS on SciServer (see the [Long pySAS Introduction](./analysis-xmm-long-intro.ipynb "Long pySAS Intro")) and that you have previously worked through the two introductory notebooks. This tutorial is partially based on the SAS thread [How to evaluate and test pile-up in an EPIC source](https://www.cosmos.esa.int/web/xmm-newton/sas-thread-epatplot).

We will be using a bright x-ray source (the star Algol) to demonstrate the pile-up effect. We will not be filtering the data, as demonstrated in previous notebooks, but will be using the unfiltered event list. Normally you should filter your data first and produce a Good Time Interval (GTI) file.

In Part 1 we breifly mentioned how to check for pile-up, which occurs whenever an X-ray source is too bright for the selected read-out mode, thus in practice is like an overexposure effect. There are two ways pile-up occurs.

    -Pattern pile-up: During a single read-out cycle more than one photon is detected in two or more adjacent pixels. During the read-out, the electronics cannot distinguish whether signals in adjacent pixels originate from one or more photons, thus two or more photons are erroneously combined to an individual event of higher pattern type, e.g. two adjacent individual photons are erroneously combined to a double event whose energy is equal to the sum of the individual energies of the incoming photons.
    
    -Energy pile-up: During a single read-out cycle more than one photon hits the same pixel. During the read-out, the electronics cannot distinguish whether the signal in a pixel originates from one or more photons, thus a single event of erroneous energy is read whose energy is equal to the sum of the individual energies of the incoming photons.

The effect of pile-up on the spectra is therefore three-fold:

    -Photon loss, either due to the fact that one photon is "read" instead of several, or to the fact that the summed energy may assume values beyond the upper energy on-board threshold. Also, an invalid pattern could be produced and hence the photons lost.
    
    -Energy distortion, whereby photons are moved to higher X-ray regions of the spectrum.
    
    -Pattern migration, where the expected pattern distribution is distorted.

#### SAS Tasks to be Used

- `evselect`[(Documentation for evselect)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/evselect/index.html)
- `epatplot`[(Documentation for epatplot)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epatplot/index.html)

#### Useful Links

- [`pysas` Documentation](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/pysas/index.html "pysas Documentation")
- [`pysas` on GitHub](https://github.com/XMMGOF/pysas)
- [Common SAS Threads](https://www.cosmos.esa.int/web/xmm-newton/sas-threads/ "SAS Threads")
- [Users' Guide to the XMM-Newton Science Analysis System (SAS)](https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/sas_usg/USG/SASUSG.html "Users' Guide")
- [The XMM-Newton ABC Guide](https://heasarc.gsfc.nasa.gov/docs/xmm/abc/ "ABC Guide")
- [XMM Newton GOF Helpdesk](https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html "Helpdesk") - Link to form to contact the GOF Helpdesk.

<div class="alert alert-block alert-warning">
    <b>Warning:</b> By default this notebook will place observation data files in your default <tt>data_dir</tt> directory. Make sure pySAS has been configured properly.
</div>

## 2. Basic Setup

In [ ]:
# pySAS imports
import pysas
from pysas.sastask import MyTask

# Useful imports
import os, subprocess

# Imports for plotting
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
plt.style.use(astropy_mpl_style)
import astropy.units as u

# To handle certain warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
obsid = '0112880701'

my_obs = pysas.obsid.ObsID(obsid)

my_obs.basic_setup(overwrite=False,repo='heasarc',rerun=False,
                   run_epproc=False,run_rgsproc=False)

os.chdir(my_obs.work_dir)

In [ ]:
# File names for this notebook. The User can change these file names.
unfiltered_event_list = my_obs.files['M1evt_list'][0]
light_curve_file = 'ltcrv.fits'
circle_selection = 'circle_selection_event_list.fits'
ring_selction1 = 'ring_selection1_event_list.fits'
ring_selction2 = 'ring_selection2_event_list.fits'
circle_epat = 'circle_epat.pdf'
ring1_epat = 'ring1_epat.pdf'
ring2_epat = 'ring2_epat.pdf'

In [ ]:
def filter_region(input_event_list,output_event_list,RA,Dec,radius,type='circle'):
    if type == 'circle':
        expression = "'((RA,DEC) in CIRCLE({0},{1},{2}))'".format(RA.value,Dec.value,radius.to(u.deg).value)
    if type == 'annulus':
        expression = "'((RA,DEC) in ANNULUS({0},{1},{2},{3}))'".format(RA.value,Dec.value,radius[0].to(u.deg).value,radius[1].to(u.deg).value)

    inargs = {'table'            : input_event_list,
              'withfilteredset'  : 'yes',
              'filteredset'      : output_event_list,
              'keepfilteroutput' : 'yes',
              'filtertype'       : 'expression',
              'expression'       : expression}
    
    MyTask('evselect', inargs).run()

In [ ]:
my_obs.quick_eplot(unfiltered_event_list)

This following cell is not necessary for what we are doing here, but is good to check that the data for this observation is generally free from flares and other contamination.

In [ ]:
my_obs.quick_lcplot(unfiltered_event_list, light_curve_file=light_curve_file)

## 3. Using `epatplot` to check for pile-up

The output of `epatplot` is a pdf. As a reminder, all files created here can be found in the work directory for this Obs ID.

In [ ]:
print(my_obs.work_dir)

First we will use a circle region to select the source. The (x,y) coordinates of the source have been determined before hand.

The following cell will extract the events inside the region using `evselect`, and write those events to a file `circle_selection_event_list.fits`. Then it will use `epatplot` to create the diagnostic plot `circle_epat.pdf`.

In [ ]:
source_RA  = 47.041508 * u.deg # degrees
source_Dec = 40.955204 * u.deg  # degrees
source_rad = 43.5 * u.arcsec # arcseconds

filter_region(unfiltered_event_list,circle_selection,source_RA,source_Dec,source_rad,type='circle')

inargs = {'set'         : circle_selection,
          'plotfile'    : circle_epat,
          'useplotfile' : 'yes',
          'pileupnumberenergyrange' : "'1000 5000'"}

MyTask('epatplot', inargs).run()

The resulting plot should look like this:
<center><img src="_files/circle_pile_up.png"/></center>

The important thing to note here is the difference between the the data (histogram) and the model (solid lines). This indicates the presence of pile-up.
<center><img src="_files/circle_pile_up_zoom.png"/></center>

To address the pile-up, instead of using a circular region we will use and annulus to cut out the very center of the bright source. We will use `evselect` again to create an event list with just the events from the annulus region, and then run that through `epatplot`. The new plot will be named `ring1_epat.pdf`.

In [ ]:
source_RA  = 47.041508 * u.deg # degrees
source_Dec = 40.955204 * u.deg  # degrees
source_rad = [8.7,43.5] * u.arcsec # arcseconds

filter_region(unfiltered_event_list,ring_selction1,source_RA,source_Dec,source_rad,type='annulus')

inargs = {'set'         : ring_selction1,
          'plotfile'    : ring1_epat,
          'useplotfile' : 'yes',
          'pileupnumberenergyrange' : "'1000 5000'"}

MyTask('epatplot', inargs).run()

The resulting plot should look like this:

<center><img src="_files/ring1_pile_up.png"/></center>

While the data is closer to the model, there is still a difference. We can improve on this by making the inner radius of the annulus larger to cut out a larger area.

In [ ]:
source_RA  = 47.041508 * u.deg # degrees
source_Dec = 40.955204 * u.deg  # degrees
source_rad = [17.4,43.5] * u.arcsec # arcseconds

filter_region(unfiltered_event_list,ring_selction2,source_RA,source_Dec,source_rad,type='annulus')

inargs = {'set' : ring_selction2,
          'plotfile' : ring2_epat,
          'useplotfile' : 'yes',
          'pileupnumberenergyrange' : "'1000 5000'"}

MyTask('epatplot', inargs).run()

The resulting plot should look like this:

<center><img src="_files/ring2_pile_up.png"/></center>

The data now closely matches the model and pile-up has been sucsessfully minimized. The final filtered event list, `ring_selection2_event_list.fits`, can now be used to extract a spectrum as shown in [ABC Guide Chapter 6, Part 2 tutorial](./analysis-xmm-ABC-guide-ch6-p2.ipynb) on filtering and extracting a spectrum.